# Video to Podcast

In [1]:
pip install -qU pytubefix moviepy openai-whisper langchain langchain-text-splitters langchain-core langchain-community lancedb langchain-openai tiktoken openai python-dotenv azure-cognitiveservices-speech

In [ ]:
from pytubefix import YouTube
from moviepy.editor import VideoFileClip
import os



# link of the video to be downloaded
# yt = YouTube('https://www.youtube.com/watch?v=8N9L-XK1eEU')
# yt = YouTube('https://www.youtube.com/watch?v=hJllkhC5GZU')
# yt = YouTube('https://www.youtube.com/watch?v=KKWPSkYN3vw')
# yt = YouTube('https://www.youtube.com/watch?v=8MMoBiIj9hI')
yt = YouTube('https://www.youtube.com/watch?v=jLVl5V8roMU')

yt.title

# Get all streams and filter for mp4 files
# yt.streams
# ... .filter(progressive=True, file_extension='mp4')
# ... .order_by('resolution')
# ... .asc()
# ... .first()
# ... .download()


'Azure AI Studio vs Copilot Studio'

# ffmpeg required for Whisper

Install ffmpeg

```
sudo apt update && sudo apt install ffmpeg
```

In [ ]:
fileName = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download()


In [ ]:
title = yt.title
url = yt.watch_url
description = yt.description
thumbnail_url = yt.thumbnail_url

# Escape description and replace newlines with \n
escaped_description = description.replace('\n', '\\n').replace('"', '\\"')

# Write a file with all the information about the video as a json
info_file_name = fileName.split('.')[0] + '_info.json'
with open(info_file_name, 'w') as f:
    f.write(f'{{"title": "{title}", "url": "{url}", "description": "{escaped_description}", "thumbnail_url": "{thumbnail_url}"}}')


In [ ]:
print(yt.description)

Azure AI Studio and Copilot Studio are different tools that enable you to create your own AI solutions. In this video I take you through the differences, how to choose the right tool for your needs, and how they can work together. 


0:00 - Azure AI Studio vs Copilot Studio
1:09 - Azure AI Studio and  Azure Open AI Studio
5:15 - Comparing Azure AI Studio and Copilot Studio
7:15 - Create a Custom Copilot in Copilot Studio using Generative Answers
11:10 - Conversational Orchestration - Creating Topics in Copilot Studio
17:27 - When to choose Azure AI Studio and when to choose Copilot Studio


-----------------------------------------------------
Connect with me:
☕ Buy me a coffee: https://www.buymeacoffee.com/lisacrosbie
🦉 Learn more about AI: https://aka.ms/learnwithlisa
🖇  LinkedIn: https://www.linkedin.com/in/lisa-crosbie/
📼 TikTok: https://www.tiktok.com/@lisa.crosbie
🐦 X (Twitter): https://twitter.com/LisaCrosbie 
📚Take my LinkedIn Learning Course: Microsoft Power Platform Fundament

In [ ]:
# Create the file name for the audio file
audioFileName = fileName.split('.')[0] + '.wav'

In [ ]:
video = VideoFileClip(fileName)
audio = video.audio
audio.write_audiofile(audioFileName)

MoviePy - Writing audio in /home/pmalarme/workspace/drop-all/Azure AI Studio vs Copilot Studio.wav


MoviePy - Done.


In [ ]:
# Cleanup: delete the video file
video.close()
os.remove(fileName)

In [ ]:
import whisper

model = whisper.load_model('base')
result = model.transcribe(audioFileName)

print(result['text'])

 If you're wondering about the difference between Azure AI Studio and Copilot Studio, or in fact, Azure Open AI Studio and the bot framework and all of these pieces that allow us to create Copilot, I'm going to take you through an explanation in a demo here to help you get this sorted out. So Azure AI Studio and Copilot Studio are actually two quite different things, and it's not necessarily one or the other they do work together. So to start with, what I'm going to do is give you a bit of a demo walkthrough of Azure AI Studio, and then we're going to spend most of the time having a look at the Copilot Studio experience so that you can get that real sense and understanding of what these tools do and how you can choose which one to use and when. So help you understand this, I'm going to frame this up using three C words, cost complexity, and another phrase called conversational orchestration. Now I reckon as we go through this video, I'm going to come up with more C words, don't worry, 

In [ ]:
# Cleanup: delete the audio file
os.remove(audioFileName)

In [ ]:
# Save the transcript
transcriptFileName = fileName.split('.')[0] + '.txt'
with open(transcriptFileName, "w") as f:
    f.write(result["text"])

In [ ]:
prompt_template = """Given title and description of a video, can you check its transcript and correct it. Give back only the corrected transcript.

Title: {title}
Description:
{description}

Transcript:
{transcript}
"""

In [ ]:
# Read .env file
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from openai import AzureOpenAI


azure_openai_client = AzureOpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
  azure_endpoint=os.environ['OPENAI_AZURE_ENDPOINT'],
  # azure_deployment=os.environ['OPENAI_AZURE_DEPLOYMENT'],
  api_version=os.environ['OPENAI_API_VERSION']
)

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
    

encoding_name = 'gpt-4o'

# Split the text in chunks of maximum 500 tokens with '.' as separator without using langchain
sentences = result['text'].split('.')
chunks = []
chunk = ''
chunk_number = 1
for sentence in sentences:
    if num_tokens_from_string(chunk + sentence, encoding_name) > 500:
        prompt = prompt_template.format(title=title, description=description, transcript=chunk)
        corrected_chunk = azure_openai_client.chat.completions.create(
            model="gpt-4o",
            temperature=0,
            top_p=1,
            messages=[
                {"role": "user", "content": prompt},
            ],
        ).choices[0].message.content
        chunks.append(corrected_chunk)
        #  # Write the file
        # chunk_file_name = fileName.split('.')[0] + '_chunk_' + str(chunk_number) + '.txt'
        # with open(chunk_file_name, "w") as f:
        #     f.write(chunk)
        # corrected_chunk_file_name = fileName.split('.')[0] + '_chunk_' + str(chunk_number) + '_corrected.txt'
        # with open(corrected_chunk_file_name, "w") as f:
        #     f.write(corrected_chunk)
        chunk = sentence + '. '
        chunk_number += 1
    else:
        chunk += sentence + '. '
        
# Write the last chunk
prompt = prompt_template.format(title=title, description=description, transcript=chunk)
corrected_chunk = azure_openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    top_p=1,
    messages=[
        {"role": "user", "content": prompt},
    ],
).choices[0].message.content
chunks.append(corrected_chunk)
# # Write the file
# chunk_file_name = fileName.split('.')[0] + '_chunk_' + str(chunk_number) + '.txt'
# with open(chunk_file_name, "w") as f:
#     f.write(chunk)
# corrected_chunk_file_name = fileName.split('.')[0] + '_chunk_' + str(chunk_number) + '_corrected.txt'
# with open(corrected_chunk_file_name, "w") as f:
#     f.write(corrected_chunk)

In [ ]:
# Create the full corrected transcript and add whitelines between the chunks but not for the last chunk
full_corrected_transcript = ''
for i, chunk in enumerate(chunks):
    full_corrected_transcript += chunk
    if i < len(chunks) - 1:
        full_corrected_transcript += '\n\n'
        

In [ ]:
# Write the full corrected transcript
full_corrected_transcript_file_name = fileName.split('.')[0] + '_corrected.txt'
with open(full_corrected_transcript_file_name, "w") as f:
    f.write(full_corrected_transcript)

In [ ]:
# Create langchain document
from langchain_core.documents.base import Document

document = Document(
  page_content=full_corrected_transcript,
  metadata={
    "title": title,
    "source": url,
    "description": description,
    "thumbnail_url": thumbnail_url
  }
)

documents = [document]  # List of documents to be processed

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200
)
splits = text_splitter.split_documents(documents)



In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
  api_key=os.environ['OPENAI_API_KEY'],
  azure_endpoint=os.environ['OPENAI_AZURE_ENDPOINT'],
  api_version=os.environ['OPENAI_API_VERSION'],
  azure_deployment=os.environ['OPENAI_AZURE_DEPLOYMENT_EMBEDDINGS']
)

In [ ]:
from langchain_community.vectorstores import LanceDB

vectorstore = LanceDB.from_documents(
  documents=splits,
  embedding=azure_openai_embeddings
)

retriever = vectorstore.as_retriever()

In [ ]:
import lancedb

db = lancedb.connect("/tmp/lancedb")
# table = db.create_table(
#     "my_table",
#     data=[
#         {
#             "vector": embeddings.embed_query("Hello World"),
#             "text": "Hello World",
#             "id": "1",
#         }
#     ],
#     mode="overwrite",
# )


In [ ]:
query = "What is LoRA?"
vectorstore.similarity_search(query)

[Document(metadata={'description': 'Azure AI Studio and Copilot Studio are different tools that enable you to create your own AI solutions. In this video I take you through the differences, how to choose the right tool for your needs, and how they can work together. \n\n\n0:00 - Azure AI Studio vs Copilot Studio\n1:09 - Azure AI Studio and  Azure Open AI Studio\n5:15 - Comparing Azure AI Studio and Copilot Studio\n7:15 - Create a Custom Copilot in Copilot Studio using Generative Answers\n11:10 - Conversational Orchestration - Creating Topics in Copilot Studio\n17:27 - When to choose Azure AI Studio and when to choose Copilot Studio\n\n\n-----------------------------------------------------\nConnect with me:\n☕ Buy me a coffee: https://www.buymeacoffee.com/lisacrosbie\n🦉 Learn more about AI: https://aka.ms/learnwithlisa\n🖇  LinkedIn: https://www.linkedin.com/in/lisa-crosbie/\n📼 TikTok: https://www.tiktok.com/@lisa.crosbie\n🐦 X (Twitter): https://twitter.com/LisaCrosbie \n📚Take my Linked

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
  azure_endpoint=os.environ['OPENAI_AZURE_ENDPOINT'],
  api_version=os.environ['OPENAI_API_VERSION'],
  azure_deployment=os.environ['OPENAI_AZURE_DEPLOYMENT']
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# response = rag_chain.invoke({"input": "What is the purpose of the video 'How to fine-tune a model using LoRA (step by step)'?"})
# response['answer']

In [ ]:
# response = rag_chain.invoke({"input": "Create an abstract of the video 'How to fine-tune a model using LoRA (step by step)'."})
# response['answer']

In [ ]:
response = rag_chain.invoke({"input": "Create an outline for a podcast based on the video " + title + "."})
print(response['answer'])

### Podcast Outline: Azure AI Studio vs Copilot Studio

#### Introduction
1. **Welcome and Introduction**
   - Brief overview of the podcast topic: Comparing Azure AI Studio and Copilot Studio.
   - Introduction of the host and any guest speakers.

2. **Episode Overview**
   - Key points to be covered:
     - Differences between Azure AI Studio and Copilot Studio.
     - Use cases for each tool.
     - How they can work together.

#### Segment 1: Understanding Azure AI Studio
1. **What is Azure AI Studio?**
   - Detailed explanation of Azure AI Studio.
   - Focus on control and customization.
   - Ideal use cases: building complex AI solutions.

2. **Features and Functions**
   - Model customization and control.
   - Building, deploying, and maintaining AI models.
   - Example scenarios where Azure AI Studio excels.

#### Segment 2: Exploring Copilot Studio
1. **What is Copilot Studio?**
   - Detailed explanation of Copilot Studio.
   - Focus on ease of use and out-of-the-box functiona

In [ ]:
# Create a prompt with the outline to get a full podcast text
podcast_outline = response['answer']
podcast_prompt = f"""Create a podcast complete text based on the following outline:

{podcast_outline}

This text will be used to generate the audio of the podcast. There are 2 participants in the podcast: the host and the guest. The host will introduce the podcast and the guest. The guest will explain the outline of the podcast. The host will ask questions to the guest and the guest will answer them. The host will thank the guest and close the podcast.
The name of the host is Pierre and his role is to be the listener's podcast assistant. The name of the guest is Marie and her role is to be the expert in the podcast topic. The name of the podcast is "Advanced AI Podcast".

When you thanks someone, write "Thank you" and the name of the person without a comma. For example, "Thank you Pierre".

Output as a JSON with the following fields:
- title: Title of the podcast
- text: an array of objects with the speaker, the intonation and the text to be spoken
Return only the json as plain text.
"""
formatted_podcast_prompt = podcast_prompt.format(podcast_outline)

podcast_script_response = rag_chain.invoke({"input": formatted_podcast_prompt})
podcast_script_text = podcast_script_response['answer']
print(podcast_script_text)

{
  "title": "Azure AI Studio vs Copilot Studio",
  "text": [
    {
      "speaker": "Pierre",
      "intonation": "enthusiastic",
      "text": "Welcome to the Advanced AI Podcast! I'm your host, Pierre, and today we have an exciting episode lined up for you. We’re diving into the world of Azure AI Studio and Copilot Studio, two powerful tools for building and managing AI solutions. Joining me is our expert guest, Marie. Welcome, Marie!"
    },
    {
      "speaker": "Marie",
      "intonation": "warm",
      "text": "Thank you Pierre! I'm thrilled to be here to discuss these two fascinating tools. We'll be comparing Azure AI Studio and Copilot Studio, exploring their key differences, use cases, and how they can work together."
    },
    {
      "speaker": "Pierre",
      "intonation": "curious",
      "text": "Great! Let's get started. Marie, can you begin by explaining what Azure AI Studio is?"
    },
    {
      "speaker": "Marie",
      "intonation": "informative",
      "text": 

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import json

# Creates an instance of a speech config with specified subscription key and service region.
speech_key = "580316191c7d4774b8c2ec0da253ef95"
service_region = "swedencentral"

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# This is an example of SSML (Speech Synthesis Markup Language) format.
# <speak version="1.0" xmlns="https://www.w3.org/2001/10/synthesis" xml:lang="en-US">
#   <voice name="en-US-AvaMultilingualNeural">
#     When you're on the freeway, it's a good idea to use a GPS.
#   </voice>
# </speak>
# Parse the JSON response and create a SSML with en-US-GuyNeural for Pierre Voice
# and en-US-JennyNeural for Marie Voice
podcast_script_json = json.loads(str(podcast_script_text))
ssml_text = "<speak version='1.0' xmlns='https://www.w3.org/2001/10/synthesis' xml:lang='en-US'>"
for line in podcast_script_json['text']:
    speaker = line['speaker']
    text = line['text']
    if speaker == 'Pierre':
        ssml_text += f"<voice name='en-US-GuyNeural'>{text}</voice>"
    elif speaker == 'Marie':
        ssml_text += f"<voice name='en-US-JennyNeural'>{text}</voice>"
ssml_text += "</speak>"

# use the default speaker as audio output.
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

result = speech_synthesizer.speak_ssml_async(ssml_text).get()
stream = speechsdk.AudioDataStream(result)
podcast_filename = fileName.split('.')[0] + '_podcast.wav'
stream.save_to_wav_file(podcast_filename)



ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
